In [1]:
import os
import sys
import numpy as np
from PIL import Image
from pytz import timezone
from pathlib import Path

In [2]:
sys.path.append('/project/modules/jmodules')
from jutils import SynJSON as SJ, get_now

In [3]:
#source data path
sdatapath = Path('/project/data/download/')
# destination data path
ddatapath = Path('/project/data/sdata3')
labelpath = ddatapath/'Label'
imagepath = ddatapath/'Image'
leftpath = imagepath/'left'
rightpath = imagepath/'right'
facepath = imagepath/'face'
os.makedirs(labelpath, exist_ok=True)
os.makedirs(facepath, exist_ok=True)
os.makedirs(leftpath, exist_ok=True)
os.makedirs(rightpath, exist_ok=True)
label_title = 'Face Left Right Origin WhichEye 3DGaze 3DHead 2DGaze 2DHead Rmat Smat GazeOrigin'

In [ ]:
%%time
final_size = 224
ratio = 1.0
half_size = int(final_size*0.5*ratio)
for nf, fold in enumerate(sdatapath.glob('fold*')):
    print(nf, fold, type(fold))
    if nf < 2:
        label_file = 'test.label'
    else:
        label_file = 'train.label'
#     label_file = f'{fold.name}.label'
    label_file = labelpath/label_file
#     lf = open(label_file, "w")
    print(label_file)
    #source
    sfoldpath = fold
    #destination
    if nf <2 :
        dfoldpath = facepath/'test'
    else:
        dfoldpath = facepath/'train' 
#     dfoldpath = facepath/fold.name
#     print(f'destination dfoldpath = {dfoldpath}')
    os.makedirs(dfoldpath, exist_ok=True)
    
    jfiles = os.listdir(sfoldpath)
    jfiles = [f for f in jfiles if f.endswith('json')]
    jfiles.sort()
    print(f"fold {fold}, n_jfiles={len(jfiles)}")
    now=get_now()
    print(now)
    lf = open(label_file, "w")
    for i, jfile in enumerate(jfiles):
        if (i+1)%200==0:
            print(f'{i+1}', end=' ')
            
        imfile = jfile.replace('info.json', 'rgb.png')
        jfile = sfoldpath/jfile
        imfile = sfoldpath/imfile
        assert imfile.is_file()
        assert jfile.is_file()
        
        # do the image file
        sj = SJ(jfile)
        face_center = sj.face_center()
        im = Image.open(imfile)
#         display(im)
        assert im.size[0] == im.size[1]
        center = face_center*im.size[0]
        center = center.astype(int)
        crop_position = np.concatenate([center-half_size, center+half_size])
        m1 = im.crop(crop_position)
        m1 = m1.resize((final_size, final_size))
        dimfilename = fold.name+'_'+str(i+1)+'.jpeg'
#         print(f'dimfilename={dimfilename}')
        dimfile = dfoldpath/dimfilename
#         print(f'dimfile={dimfile}')
        m1.save(dimfile)
 
        #write label:
        
#         face = '/'.join(dimfile.split('/')[-3:])
        face = dimfile.relative_to('/project/data/sdata3/Image/').as_posix()
        left = 'NA'
        right = 'NA'
#         origin = '/'.join(imfile.split('/')[-2:])
        origin = imfile.relative_to('/project/data/download/').as_posix()
#         print(f'face={face}')
#         print(f'origin={origin}')
#         pitch, yaw = sj.pitch_yaw()
        pitch, yaw = sj.pitchyaw2d()

        pitch, yaw = sj.pitchyaw2d()  #in radian and average
#         print(pitchyaw)
        whicheye = 'NA'
        d3gaze = '0.0,0.0,0.0'
        d3head = '0.0,0.0,0.0'
        d2gaze = f'{pitch},{yaw}'
        d2head = '0.0,0.0'
        rmat=smat = '1.0,1.0,1.0'
        gazeorigin = '0.0,0.0,0.0'
        to_print = ' '.join([face, left, right, origin, whicheye, d3gaze, d3head, d2gaze, d2head, rmat, smat, gazeorigin])
#         print(to_print)
        print(to_print, file=lf)
        
    lf.close()
        

0 /project/data/download/fold03 <class 'pathlib.PosixPath'>
/project/data/sdata3/Label/test.label
fold /project/data/download/fold03, n_jfiles=21511
07/21/2022 18:06:02
200 400 600 800 1000 1200 1400 1600 1800 2000 2200 2400 2600 2800 3000 3200 3400 3600 3800 4000 4200 4400 4600 4800 5000 5200 5400 5600 5800 

In [ ]:
pp = Path('/project/data/sdata3/Image/face/test/fold03_1.jpeg')

In [ ]:
pp.relative_to('/project/data/sdata3/Image/').as_posix()